In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_PROJECT'] = 'advanced-rag'
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")
os.environ['GROQ_API_KEY'] = os.getenv("GROQQ_API_KEY")

In [114]:
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
template = """You are a helpful assistant that translates the comment passed in Indian languages to their original language phrase. 
Please give only original language phrase as shown in example
Eg : comment : "mere naam hai Roshan" , original language phrase : "मेरा नाम है रोशन"
 comment : "nenu UST lo Panichestunna" , original language phrase : "nenu UST లో పనిచేస్తున్నా"
{comment} \n
original language phrase: Output"""

prompt = ChatPromptTemplate.from_template(template)

generate = (
    prompt
    | ChatGroq(temperature=0)
    | StrOutputParser()
    | (lambda x: x.split("\n")) )


In [115]:
comment= "mere naam kya hai bhai"
generate.invoke({"comment" : comment})[0]

' "मेरा नाम क्या है भाई"'

In [116]:
comment= "naa peru dhanunjai"
generate.invoke({"comment" : comment})[0]

'"నా పేరు ధనుంజై"'

In [117]:
comment= "chilappol suryanum bhoomiye chuttunnu"
generate.invoke({"comment" : comment})[0]

'"சிலப்பொல் சூர்யனும் பூமியே சுட்டுன்னு"'